In [ ]:
import os 
import numpy as np 
import pandas as pd
from collections import defaultdict
import itertools
import ast
import csv

In [5]:
#hyper parameters
rating_threshold = 4.5 #only counted the usr rating if it is rated above the rating threshold
user_count_threshold = 500 #only keep the bk_id if the bk is rated by user for more than user_count_threshold times
split = [0.8, 0.1, 0.1] #train, dev, test split
#INPUT_DIR = '../../../data/the-movies-dataset/'
INPUT_DIR ='../../data/goodbooks-10k-master/'
OUTPUT_DIR = '../box-code/data/book_data/big/taxonomy/'

In [6]:
#read in external taxonomy
tax_file = INPUT_DIR + 'ext_genres_taxo.csv'
df_ex_tax = pd.read_csv(tax_file, delimiter=',')
df_ex_tax.dataframeName = 'ex_tax.csv'
print(df_ex_tax.head(10))

                     Parent                                            Child
0              10th-century                        ['Fiction', 'Literature']
1              11th-century                        ['Fiction', 'Literature']
2              12th-century                        ['Fiction', 'Literature']
3              13th-century                        ['Fiction', 'Literature']
4              14th-century                        ['Fiction', 'Literature']
5              15th-century  ['Fiction', 'Historical Fiction', 'Literature']
6              16th-century                        ['Fiction', 'Literature']
7              17th-century  ['Fiction', 'Historical Fiction', 'Literature']
8  1864-shenandoah-campaign       ['American History', 'American Civil War']
9              18th-century  ['Fiction', 'Historical Fiction', 'Literature']


In [8]:
#preprocess external taxonomy
for i, col in enumerate(df_ex_tax.columns):
    df_ex_tax.iloc[:, i] = df_ex_tax.iloc[:, i].str.replace('"', '')
    df_ex_tax.iloc[:, i] = df_ex_tax.iloc[:, i].str.replace('\'', '')
    df_ex_tax.iloc[:, i] = df_ex_tax.iloc[:, i].str.replace('[', '')
    df_ex_tax.iloc[:, i] = df_ex_tax.iloc[:, i].str.replace(']', '')

In [9]:
print(df_ex_tax.head(10))

                     Parent                                    Child
0              10th-century                      Fiction, Literature
1              11th-century                      Fiction, Literature
2              12th-century                      Fiction, Literature
3              13th-century                      Fiction, Literature
4              14th-century                      Fiction, Literature
5              15th-century  Fiction, Historical Fiction, Literature
6              16th-century                      Fiction, Literature
7              17th-century  Fiction, Historical Fiction, Literature
8  1864-shenandoah-campaign     American History, American Civil War
9              18th-century  Fiction, Historical Fiction, Literature


In [11]:
import string
def strProcess(s):
    if not s: return 
    s = s.replace('\\u200', '')
    s = s.lstrip().lower().replace(' ', '-')
    if not s.replace('-', '').isalnum():
        #print(s)
        return
    return s

In [12]:
#read external taxonomy into dict
ex_tax_dict={} #key: term 1 in external taxonomy, val: term 2 in external taxonomy(a list)
for index, row in df_ex_tax.iterrows():
    child = row['Child'].split(",")
    parent = row['Parent'].replace(' ', '-')
    for i in child:
        i = strProcess(i)
        if not i: continue
        if parent not in ex_tax_dict:
            ex_tax_dict[parent] = [i]
        else:
            ex_tax_dict[parent].append(i)

In [15]:
#for k, v in ex_tax_dict.items():
 #   print(k,v)

In [30]:
#save ex_tax_dict
import pickle
with open('objs.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump(ex_tax_dict, f)

In [ ]:
'''
#getting back ex_tax_dict
with open('objs.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
    ex_tax_dict = pickle.load(f)
'''